# Introduction

## Project Objectives
The analysis will focus on the number of matches won. This project is divided into two key objectives:

1. Analysis of relevant parameters:
- General performance of the teams.
- Team with the most matches won per tournament.
- Team with the most goals scored.
- Team with the most matches won historically.
  
2. Creating a new dataset to develop an interactive Tableau dashboard.

A dataset that is too extensive can be problematic for visualization in tools like Tableau. To optimize this process, a new dataset is created and made available for anyone who may need it.

## Dataset Description
We will analyze the various competitions included in the dataset related to Argentine football clubs, focusing on tournaments from the 1970s to 2020.

While all variables in the dataset will be reviewed, we will highlight the **key variables** for this analysis:
- **data_name**
- **local_team**
- **local_result**
- **visitor_result**
- **visitor_team**

## Libraries Used
We will use the following Python libraries:
- pandas
- seaborn
- matplotlib
- numpy

# Coding

## Importing Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

## General Overview of the Dataset

In [ ]:
data = pd.read_csv('/kaggle/input/liga-argentina-futbol/results.csv')
data.head()

In [ ]:
data.info()

We can observe the variables in our dataset, including the key variables. Additionally, the dataset has no "null" values, saving us the effort of cleaning or imputing data.

## Initial Conclusions

In [ ]:
# Number of teams
num_team = data['local_team_id'].nunique()
print(f'There are {num_team} unique teams in this dataset')
print('---' )

# Team with the most goals at home
max_score_local = data['local_result'].max()
team_max_score_local = data.loc[data['local_result']== max_score_local, 'local_team'].iloc[0]
date_max_score_local = data.loc[data['local_result']== max_score_local, 'date_name'].iloc[0]
vs_max_score_local = data.loc[data['local_result']== max_score_local, 'visitor_team'].iloc[0]
print(f'The team with the highest number of goals scored at home is {team_max_score_local}, with {max_score_local} goals against {vs_max_score_local} during {date_max_score_local}')
print('---')

# Team with the most goals at visitors
max_score_visitor = data['visitor_result'].max()
team_max_score_visitor = data.loc[data['visitor_result']== max_score_visitor, 'visitor_team'].iloc[0]
date_max_score_visitor = data.loc[data['visitor_result']== max_score_visitor, 'date_name'].iloc[0]
vs_max_score_local = data.loc[data['visitor_result']== max_score_visitor, 'local_team'].iloc[0]
print(f'The team with the highest number of goals scored as visitors is {team_max_score_visitor}, with {max_score_visitor} goals against {vs_max_score_local} during {date_max_score_visitor}')

- The dataset includes 95 different teams.
- Argentinos Juniors is the home team with the most goals scored.
- Independiente is the away team with the most goals scored.

## Dataset Modification
The column "data_name" contains critical information for our analysis, such as the tournament to which each match belongs. We will create two new columns:
- "tournament": contains the tournament information.
- "detail": contains the remaining information from the cell.

In [ ]:
# 1st modification
data['tournament'] = data['date_name'].str.split('-').str[0]
data['detail'] = data['date_name'].str.split('-').str[1]
data.head()

To simplify the investigation of the team with the most match wins, we will add a new column called "winner", identifying the winning team for each match.

In [ ]:
# 2nd modification
data['winner'] = data.apply(lambda row: row['local_team'] if row['local_result'] > row['visitor_result'] else row['visitor_team'], axis=1)
data.head()

## Secondary Conclusions

In [ ]:
# Which team has the most historical wins?
history_winner = data['winner'].mode()[0]
print (f' Historically, the team with the most match wins is {history_winner}')

- River Plate is the team with the most matches won historically, according to the dataset.

## New Dataset
At this point, we will create the new dataset for Tableau visualization.

The objective is to determine how many times a team has been the one with the most matches won in tournaments.

In [ ]:
# New dataframe
data_winner = (data.groupby(['tournament','winner']).size().reset_index(name = 'matches_won'))
data_winner = data_winner.loc[data_winner.groupby('tournament')['matches_won'].idxmax()]
data_winner = data_winner.reset_index(drop = True)
print (data_winner)

In [ ]:
# Who's the best?
most_winner = data_winner['winner'].mode()[0]
game_most_winner = data_winner['winner'].value_counts().max()
print(f'In {game_most_winner} out of 136 competitions, {most_winner} was the team with the most matches won. This number represents {((27/136)*100):.2f}% of all competitions')


Using the mode of the new dataset, we observe that Boca Juniors has been the team with the most match wins in 27 out of 136 competitions.

However, the mode only considers the first repeated value. To better compare results across teams, we will create a table for clearer visualization.

In [ ]:
# Table
graph_most_winner = data_winner['winner'].value_counts()
print (graph_most_winner)

Indeed, River Plate also meets the aforementioned condition.

## General Barplot
We will present a bar chart to visualize the overall performance of the clubs across the different competitions.

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=graph_most_winner.index, y=graph_most_winner.values)
plt.title('Number of times a team has won the Most Matches in a Tournament')
plt.xlabel('Teams')
plt.ylabel('Times a team has benn the top Winner')
plt.xticks(rotation=45)
plt.show()

## Final Conclusions
Based on our analysis, we can draw the following conclusions:

- The home team with the most goals scored is Argentinos Juniors (12 goals against Talleres [C] in the 1986/87 Championship - Matchday 16).
- The away team with the most goals scored is Independiente (11 goals against Platense in the 1971 Metropolitano - Matchday 2).
- River Plate is the team with the most matches won historically.
- Both Boca Juniors and River Plate were the teams with the most matches won in 27 competitions each, out of 136 total.
- This statistic represents almost 40% of all tournaments, a dominance clearly visible in the bar chart.
- The third team, Independiente, just won 12 matches compared to Boca Juniors and River Plate who's won 27.

## Modified Dataset for Tableau
Below is the dataset used in Tableau for the dashboard visualization.

In [ ]:
data_winner.to_csv('/kaggle/working/data_winner.csv', index=False)